In [33]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [34]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Scrape New Completed Games and Format**

In [35]:

def get_new_games(driver)-> pd.DataFrame:

    # set search strings for the last seven days 
    DAYS = 7
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")

    # initate a webdriver in selenium 
    # since website data is dynamically generated
    driver = activate_web_driver('firefox')

    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    driver.close() 

    df = convert_columns(df)
    df = combine_home_visitor(df)

    return df

df_new = get_new_games(driver)

df_new


2022-12-10 12:29:01,651 INFO: ====== WebDriver manager ======
2022-12-10 12:29:01,943 INFO: Get LATEST geckodriver version for 107.0 firefox
2022-12-10 12:29:02,827 INFO: Driver [C:\Users\Chris\.wdm\drivers\geckodriver\win64\0.32\geckodriver.exe] found in cache


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-12-09,1,121,44.1,38.2,86.7,48,25,1610612752,22200377,102,44.0,23.1,78.6,45,16,1610612766,2022
1,2022-12-09,0,103,40.0,31.4,74.1,47,23,1610612765,22200383,114,48.4,28.1,53.1,48,30,1610612763,2022
2,2022-12-09,0,122,50.0,35.5,76.7,37,29,1610612747,22200382,133,53.3,45.0,86.4,42,30,1610612755,2022
3,2022-12-09,1,106,53.0,48.5,62.5,38,21,1610612758,22200381,95,43.0,33.3,77.3,36,20,1610612739,2022
4,2022-12-09,0,117,50.0,50.0,69.2,37,30,1610612756,22200384,128,51.1,29.6,80.0,44,27,1610612740,2022
5,2022-12-09,0,111,53.2,34.5,76.0,38,25,1610612764,22200378,121,50.6,43.9,85.0,40,23,1610612754,2022
6,2022-12-09,1,106,43.7,34.4,82.6,51,27,1610612749,22200386,105,46.6,32.5,41.7,43,21,1610612742,2022
7,2022-12-09,0,116,49.5,34.5,87.5,38,21,1610612737,22200380,120,54.2,43.3,81.0,38,30,1610612751,2022
8,2022-12-09,0,109,42.4,28.6,81.8,33,23,1610612761,22200379,113,59.2,38.5,90.5,43,24,1610612753,2022
9,2022-12-09,1,118,53.2,40.0,76.9,44,30,1610612750,22200385,108,43.3,29.5,70.8,40,22,1610612762,2022


**Retrieve todays games**

In [36]:
#retrieve list of teams playing today

# get today's games on NBA schedule
teams_list = get_todays_matchups(driver)

team_count = len(teams_list) 
matchups = []
for i in range(0,team_count,2):
    visitor_id = teams_list[i].partition("team/")[2].partition("/")[0] #extract team id from text
    home_id = teams_list[i+1].partition("team/")[2].partition("/")[0]
    matchups.append([visitor_id, home_id])

matchups

[['1610612759', '1610612748'],
 ['1610612751', '1610612754'],
 ['1610612746', '1610612764'],
 ['1610612760', '1610612739'],
 ['1610612742', '1610612741'],
 ['1610612738', '1610612744'],
 ['1610612762', '1610612743'],
 ['1610612750', '1610612757']]

**Create Blank Rows**

In [37]:
# append today's matchups to the new games dataframe

# since we don't have access to official game ids, we will use the index as a game id
# this data is only used for prediction and is deleted after the prediction is made
for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 'VISITOR_TEAM_ID': matchup[0], 'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 'GAME_ID': i}
    game_details_df = pd.DataFrame(game_details, index=[i])
    df_new = pd.concat([df_new, game_details_df], ignore_index = True)

df_new


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-12-09 00:00:00,1.0,121.0,44.1,38.2,86.7,48.0,25.0,1610612752,22200377,102.0,44.0,23.1,78.6,45.0,16.0,1610612766,2022.0
1,2022-12-09 00:00:00,0.0,103.0,40.0,31.4,74.1,47.0,23.0,1610612765,22200383,114.0,48.4,28.1,53.1,48.0,30.0,1610612763,2022.0
2,2022-12-09 00:00:00,0.0,122.0,50.0,35.5,76.7,37.0,29.0,1610612747,22200382,133.0,53.3,45.0,86.4,42.0,30.0,1610612755,2022.0
3,2022-12-09 00:00:00,1.0,106.0,53.0,48.5,62.5,38.0,21.0,1610612758,22200381,95.0,43.0,33.3,77.3,36.0,20.0,1610612739,2022.0
4,2022-12-09 00:00:00,0.0,117.0,50.0,50.0,69.2,37.0,30.0,1610612756,22200384,128.0,51.1,29.6,80.0,44.0,27.0,1610612740,2022.0
5,2022-12-09 00:00:00,0.0,111.0,53.2,34.5,76.0,38.0,25.0,1610612764,22200378,121.0,50.6,43.9,85.0,40.0,23.0,1610612754,2022.0
6,2022-12-09 00:00:00,1.0,106.0,43.7,34.4,82.6,51.0,27.0,1610612749,22200386,105.0,46.6,32.5,41.7,43.0,21.0,1610612742,2022.0
7,2022-12-09 00:00:00,0.0,116.0,49.5,34.5,87.5,38.0,21.0,1610612737,22200380,120.0,54.2,43.3,81.0,38.0,30.0,1610612751,2022.0
8,2022-12-09 00:00:00,0.0,109.0,42.4,28.6,81.8,33.0,23.0,1610612761,22200379,113.0,59.2,38.5,90.5,43.0,24.0,1610612753,2022.0
9,2022-12-09 00:00:00,1.0,118.0,53.2,40.0,76.9,44.0,30.0,1610612750,22200385,108.0,43.3,29.5,70.8,40.0,22.0,1610612762,2022.0


**Access Feature Store**

In [38]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350
Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [39]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [40]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


2022-12-10 12:36:02,818 INFO: USE `nba_predictor_featurestore`
2022-12-10 12:36:03,169 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2007-03-12,20600946,1610612756,1610612745,2006,103,0.500000,0.727051,0.600098,18,50,82,0.385986,0.722168,0.262939,13,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22592,2022-12-07,22200372,1610612744,1610612762,2022,123,45.906250,76.000000,33.312500,26,42,124,53.500000,81.000000,42.906250,26,40,0
22593,2022-12-05,22200352,1610612746,1610612766,2022,119,48.406250,66.687500,41.687500,30,47,117,48.000000,64.687500,35.312500,29,49,1
22594,2022-12-03,22200340,1610612753,1610612761,2022,108,48.687500,80.812500,39.312500,23,29,121,56.000000,72.000000,31.000000,31,43,0
22595,2022-12-03,22200339,1610612749,1610612766,2022,105,47.000000,78.875000,37.500000,24,47,96,39.093750,88.500000,20.796875,15,44,1


**Convert Feature Names back to original mixed case**

In [41]:
df_old = convert_feature_names(df_old)
df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2007-03-12,20600946,1610612756,1610612745,2006,103,0.500000,0.727051,0.600098,18,50,82,0.385986,0.722168,0.262939,13,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22592,2022-12-07,22200372,1610612744,1610612762,2022,123,45.906250,76.000000,33.312500,26,42,124,53.500000,81.000000,42.906250,26,40,0
22593,2022-12-05,22200352,1610612746,1610612766,2022,119,48.406250,66.687500,41.687500,30,47,117,48.000000,64.687500,35.312500,29,49,1
22594,2022-12-03,22200340,1610612753,1610612761,2022,108,48.687500,80.812500,39.312500,23,29,121,56.000000,72.000000,31.000000,31,43,0
22595,2022-12-03,22200339,1610612749,1610612766,2022,105,47.000000,78.875000,37.500000,24,47,96,39.093750,88.500000,20.796875,15,44,1


**Combine New Data with Old Data**

In [42]:
df_combined = pd.concat([df_new, df_old])
df_combined

,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-12-09 00:00:00,1.0,121.0,44.10000,38.20000,86.7000,48.0,25.0,1610612752,22200377,102.0,44.00000,23.100000,78.6000,45.0,16.0,1610612766,2022.0
1,2022-12-09 00:00:00,0.0,103.0,40.00000,31.40000,74.1000,47.0,23.0,1610612765,22200383,114.0,48.40000,28.100000,53.1000,48.0,30.0,1610612763,2022.0
2,2022-12-09 00:00:00,0.0,122.0,50.00000,35.50000,76.7000,37.0,29.0,1610612747,22200382,133.0,53.30000,45.000000,86.4000,42.0,30.0,1610612755,2022.0
3,2022-12-09 00:00:00,1.0,106.0,53.00000,48.50000,62.5000,38.0,21.0,1610612758,22200381,95.0,43.00000,33.300000,77.3000,36.0,20.0,1610612739,2022.0
4,2022-12-09 00:00:00,0.0,117.0,50.00000,50.00000,69.2000,37.0,30.0,1610612756,22200384,128.0,51.10000,29.600000,80.0000,44.0,27.0,1610612740,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22592,2022-12-07 00:00:00,0.0,123.0,45.90625,33.31250,76.0000,42.0,26.0,1610612744,22200372,124.0,53.50000,42.906250,81.0000,40.0,26.0,1610612762,2022.0
22593,2022-12-05 00:00:00,1.0,119.0,48.40625,41.68750,66.6875,47.0,30.0,1610612746,22200352,117.0,48.00000,35.312500,64.6875,49.0,29.0,1610612766,2022.0
22594,2022-12-03 00:00:00,0.0,108.0,48.68750,39.31250,80.8125,29.0,23.0,1610612753,22200340,121.0,56.00000,31.000000,72.0000,43.0,31.0,1610612761,2022.0
22595,2022-12-03 00:00:00,1.0,105.0,47.00000,37.50000,78.8750,47.0,24.0,1610612749,22200339,96.0,39.09375,20.796875,88.5000,44.0,15.0,1610612766,2022.0


**Data Processing**

In [43]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON,PLAYOFF,TARGET
0,2022-12-09 00:00:00,1.0,121.0,44.100000,38.200000,86.700000,48.0,25.0,1610612752,22200377,102.0,44.000000,23.100000,78.600000,45.0,16.0,1610612766,2022.0,0,1.0
1,2022-12-09 00:00:00,0.0,103.0,40.000000,31.400000,74.100000,47.0,23.0,1610612765,22200383,114.0,48.400000,28.100000,53.100000,48.0,30.0,1610612763,2022.0,0,0.0
2,2022-12-09 00:00:00,0.0,122.0,50.000000,35.500000,76.700000,37.0,29.0,1610612747,22200382,133.0,53.300000,45.000000,86.400000,42.0,30.0,1610612755,2022.0,0,0.0
3,2022-12-09 00:00:00,1.0,106.0,53.000000,48.500000,62.500000,38.0,21.0,1610612758,22200381,95.0,43.000000,33.300000,77.300000,36.0,20.0,1610612739,2022.0,0,1.0
4,2022-12-09 00:00:00,0.0,117.0,50.000000,50.000000,69.200000,37.0,30.0,1610612756,22200384,128.0,51.100000,29.600000,80.000000,44.0,27.0,1610612740,2022.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22542,2017-12-10 00:00:00,1.0,126.0,0.526855,0.399902,0.817871,40.0,24.0,1610612754,21700387,116.0,0.452881,0.405029,0.812988,47.0,24.0,1610612743,2017.0,0,1.0
22543,2015-04-10 00:00:00,0.0,99.0,0.460938,0.350098,1.000000,46.0,21.0,1610612753,21401175,101.0,0.415039,0.300049,0.933105,46.0,22.0,1610612761,2014.0,0,0.0
22544,2005-01-15 00:00:00,1.0,73.0,0.333008,0.399902,0.730957,49.0,15.0,1610612745,20400537,67.0,0.353027,0.125000,0.629883,39.0,10.0,1610612759,2004.0,0,1.0
22545,2012-03-07 00:00:00,0.0,104.0,0.477051,0.333008,0.789062,35.0,29.0,1610612749,21100575,106.0,0.518066,0.312988,0.881836,43.0,26.0,1610612741,2011.0,0,0.0


**Feature Engineering**

In [44]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28,1.0,109,0.505859,0.350098,0.600098,46,32,1610612747,20300003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28,1.0,89,0.439941,0.350098,0.533203,39,25,1610612755,20300001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28,1.0,83,0.425049,0.099976,0.769043,38,20,1610612759,20300002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29,1.0,88,0.323975,0.160034,0.700195,55,24,1610612740,20300006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29,0.0,87,0.391113,0.238037,0.587891,51,19,1610612744,20300013,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22592,2022-12-09,0.0,111,53.187500,34.500000,76.000000,38,25,1610612764,22200378,...,-5.870654,-3.899772,0.666667,1.285714,2.3,0.266667,-1.666667,-3.142857,-4.1,-4.333333
22593,2022-12-09,0.0,109,42.406250,28.593750,81.812500,33,23,1610612761,22200379,...,0.182153,0.115177,3.666667,-1.571429,-2.5,-2.200000,3.000000,1.571429,0.8,0.000000
22594,2022-12-09,0.0,117,50.000000,50.000000,69.187500,37,30,1610612756,22200384,...,3.959998,2.645532,1.333333,1.142857,1.2,3.000000,3.000000,-0.571429,-4.3,-2.066667
22595,2022-12-09,1.0,118,53.187500,40.000000,76.875000,44,30,1610612750,22200385,...,-1.072192,-0.730721,1.333333,2.714286,4.1,2.933333,4.333333,1.000000,-2.1,-2.666667


**Insert New Data into Feature Group**

In [45]:

# retrieve only new games from the combined dataframe

# set index to GAME_ID
df_combined = df_combined.set_index('GAME_ID')
df_new = df_new.set_index('GAME_ID')
 
# retrieve only new games
df_new = df_combined.loc[df_new.index]

# reset GAME_ID index back to column
df_new = df_new.reset_index()

# convert GAME_ID back to int32 for Hopsworks compatibility
df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')

# save new games to Hopsworks feature group
rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

df_new 



KeyError: '[0, 1, 2, 3, 4, 5, 6, 7] not in index'